In [ ]:
!git clone https://github.com/woonkirov/KnuSentiLex

Cloning into 'KnuSentiLex'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.


In [ ]:
import pandas as pd
from collections import Counter
import ast
import numpy as np
from google.colab import files
from sklearn.model_selection import train_test_split

In [ ]:
knu_lexicon = pd.read_json('/content/KnuSentiLex/data/SentiWord_info.json', orient='records')
knu_lexicon

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14849,반신반의하다,반신반의,0
14850,신비롭다,신비,1
14851,아리송하다,아리송,-1
14852,알쏭하다,알쏭하,-1


In [ ]:
df = pd.read_csv('DB_part2.csv', encoding = 'utf-8')
#df = df.sample(frac=1).reset_index(drop=True)
df['score'] = None
df['posneg'] = np.nan
df

In [ ]:
words_count = Counter()

df.del_stopwords = df.del_stopwords.apply(ast.literal_eval) # 런타임 초기화 및 재실행시 tuple, list를 인식하지 못하고 한 글자씩 인식하는 문제가 발생. 이 문제를 해결하기 위한 코드 # import ast
df['del_stopwords'].apply(lambda x: words_count.update(x))
words_count.most_common(), len(words_count)

In [ ]:
custom_lexicon = []

for token in list(words_count):
  for idx, word in enumerate(knu_lexicon.word):
    if token[0] == word:
      custom_polarity = (token[0], knu_lexicon.polarity[idx])
      custom_lexicon.append(custom_polarity)

custom_lexicon

In [ ]:
# for i in list(words_count.keys()):
#   if i[1] == 'Noun':
#     print(i)

In [ ]:
# for i in list(words_count.keys()):
#   if i[1] == 'KoreanParticle':
#     print(i)

In [ ]:
for idx, tokens in enumerate(df.del_stopwords):
  score_list = []
  score_list.append(0)

  for token in tokens:
    for i in custom_lexicon:
      if token[0] == i[0]:
        score_list.append(i[1])
    df.score[idx] = score_list
    df.posneg[idx] = sum(score_list)

df

In [ ]:
df['label'] = np.nan

for idx, score in enumerate(df.posneg):
  if score == 0:
    df.label[idx] = 2
  elif score < 0:
    df.label[idx] = 0
  else:
    df.label[idx] = 1

df['label'] = df['label'].astype(int)
df

In [ ]:
temp = pd.DataFrame(df)
temp.to_csv('DB_labeling.csv', encoding = 'utf-8-sig', index = False)
files.download('DB_labeling.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_raw = pd.read_csv('DB_labeling.csv')
df_raw

In [ ]:
df_pos = df.loc[df['label'] == 1][['', '', '', '', '', '']]
df_neg = df.loc[df['label'] == 0][['', '', '', '', '', '']]
# df_neu = df.loc[df['label'] == '1'][['', '', '', '', '', '']]

df_pos = df_pos.sample(n=7500, random_state = 42, replace=True)
df_neg = df_neg.sample(n=7500, random_state = 42, replace=True)
# df_neu = df_neu.sample(n=5000, random_state = 42, replace=True)

df_train = pd.concat([df_pos, df_neg]).reset_index(drop=True)
df_train['label']
train = df_train[['index', 'message', 'label']]
train

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(train, train_size=0.80, test_size=0.20, stratify=train['label'], random_state=42)
train.shape, test.shape

((9600, 3), (2400, 3))

In [ ]:
temp_train = pd.DataFrame(train)
temp_test = pd.DataFrame(test)

temp_train.to_csv('DB_train.csv', encoding = 'utf-8-sig', index=False)
temp_test.to_csv('DB_test.csv', encoding = 'utf-8-sig', index=False)

files.download('DB_train.csv')
files.download('DB_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>